# Tuning the hyperparameter for graph construction

What else do we have left?

Let's take the algorithms as fixed.

We need to construct our evaluation cases. But we first need to tune the hyperparameters of each method to make it fair.

Our objectives:
- The graph cannot be overly dense
- Each node must have at least 1 edge
- The graph has a decent modularity (there are branches and such), on at least the same order of magnitude as the number of documents.

In [1]:
import os
import sys
import pickle
import networkx as nx
import matplotlib as mpl
import numpy as np
import random
import matplotlib.pyplot as plt
import re
from pprint import pprint

# Add the project root directory to the system path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

from StructuredRag.evaluation import graph_scoring
from StructuredRag.processing import graph_construction
from StructuredRag.algorithms import v0, v1
from StructuredRag.processing import distance_metrics

import hyperopt
import joblib

/Users/lukasalemu/Downloads/ls/envs/dissertation_rag/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


## Load data

In [2]:
# Select the runs you want to load
for experiment in sorted(os.listdir('../results')):
    print('Experiment:', experiment)
    for run in sorted(os.listdir('../results/' + experiment)):
        print("     || Run:", run)

Experiment: v0
     || Run: 2024-05-10
     || Run: 2024-05-14
     || Run: 2024-05-25
     || Run: 2024-05-27
Experiment: v1
     || Run: 2024-05-10
     || Run: 2024-05-14
Experiment: v3
     || Run: 2024-05-10
     || Run: 2024-05-14
Experiment: v4
     || Run: 2024-05-10
     || Run: 2024-05-14
Experiment: v5
     || Run: 2024-05-14


In [5]:
run_path = 'v0/2024-05-27'

data = {}
# Read the data for the specified experiment
for item in os.listdir('../results/' + run_path):
    print('Loading item:', item.split('.')[0])
    
    with open('../results/' + run_path + '/' + item, 'rb') as f:
        data[item.split('.')[0]] = pickle.load(f)

Loading item: embedded_index
Loading item: edge_thresh
Loading item: adj_matrix


In [6]:
for d in data['adj_matrix'].values():
    for d2 in d.values():
        print(d2['weight'])
        break


0.41583253022146693
0.37284286780664944
0.48331059636215135
0.6937518041319345
0.7305841757076126
0.4079109142376651
0.8772880394205684
0.45831245228687645
0.5750168356229446
0.6345638318416553
0.7051192399687273
0.576172779971392
0.5623051504032051
0.7054687085888783
0.6138098535317034
0.5891259230809329
0.654588652754142
0.5901699092587834
0.439464956193647
0.5588657716680172
0.6798129118900114
0.5073503441307814
0.4907973016041458
0.3292784998623075
0.5968640802102971
0.3845128205362731
0.3509131171704657
0.4730268620706116
0.5374893384690567
0.6081984588288654
0.6037924862916089
0.5644579590292235
0.6761095904387218
0.3129854131661006
0.8006043994276713
0.7581028371444745
0.15642860642222442
0.782166537736738
0.484719437202122
0.1538627371660964
0.33297026213592507
0.3811410690934725
0.1791019075488202
0.36812193747635485
0.30696570560747527
0.20183402182368718
0.384108311063049
0.3317061206151368
0.6193628501232397
0.6096030581262905
0.46178736387689945
0.4526447408092249
0.491771

# New

In [7]:
edge_weights = [d2['weight'] for d in data['adj_matrix'].values() for d2 in d.values()]

In [10]:
len(edge_weights)

3687

In [13]:
len(data['adj_matrix'].keys())

257

In [24]:
search_space = np.linspace(np.min(edge_weights), np.max(edge_weights), 50)

In [28]:
results = []
for n in search_space:
    proportion_above_thresh = len([x for x in edge_weights if x > n]) / len(edge_weights)
    # print(f"Threshold: {n}, Proportion above threshold: {proportion_above_thresh}")
    results.append((n, proportion_above_thresh))
    
# Find nearest where proportion above threshold is 0.2
nearest = min(results, key=lambda x: abs(x[1] - 0.2))
print(nearest)


(0.5401552621973181, 0.20585842148087877)


In [9]:
np.mean(edge_weights)

0.43281203899811366

# Old - using Hyperopt

## Tune Hyperparams

In [35]:
# To avoid writing boilerplate, lets use hyperopt

# Input to our tuning
def objective(params):
    """This is our function to return the score to minimise based on our input
    Hyperopt minimises the objective function by default.

    Args:
        params (_type_): _description_
    """

    graph = graph_construction.construct_graph_from_adj_dict(
        adj_dict=data['adj_matrix'],
        edge_thresh=params['edge_thresh'],
        embedded_index=data['embedded_index']
    )

    try:
        graph_scores = graph_scoring.quick_stats(graph)

        # I want average degree to be around 10. Square to penalise large differences much more
        # Expect this to be around 10 max once it hits equilibrium
        avg_deg_penalty = (10 - graph_scores['average_degree'])**2
        
        # Need to make sure every node is connected
        if not nx.is_connected(graph):
            connected_penalty = 1000
        else:
            connected_penalty = 0
        
        # Try to encourage our graph having some decent modularity
        graph_modularity = graph_scoring.community_detection(graph)

        # Higher is better, but not super emphasized. So make it negative and *10.
        modularity_penalty = -(graph_modularity['community_modularity'])*10

        return (avg_deg_penalty + connected_penalty + modularity_penalty)
    
    except ZeroDivisionError:
        return 10000


search_space = {
    'edge_thresh': hyperopt.hp.uniform('edge_thresh', -1, 2)
}

trials = hyperopt.Trials()

with joblib.parallel_backend('loky', n_jobs=1):
    best_params = hyperopt.fmin(
        fn=objective, 
        space=search_space, 
        algo=hyperopt.tpe.suggest, 
        max_evals=100, 
        trials=trials
    )

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

Calculating quick stats                                
Calculating community detection                        
Calculating quick stats                                               
Calculating community detection                                       
  1%|          | 1/100 [00:01<00:19,  5.02trial/s, best loss: 10000.0]

job exception: dictionary update sequence element #0 has length 1; 2 is required



  1%|          | 1/100 [00:06<10:43,  6.50s/trial, best loss: 10000.0]


ValueError: dictionary update sequence element #0 has length 1; 2 is required

In [ ]:
best_params

{'edge_thresh': 0.6074136091301304}

In [ ]:
graph = graph_construction.construct_graph_from_adj_dict(
        adj_dict=data['adj_matrix'],
        edge_thresh=best_params['edge_thresh'],
        # edge_thresh=0.95,
        embedded_index=data['embedded_index']
)

print(graph_scoring.quick_stats(graph))
print(graph_scoring.community_detection(graph)['community_modularity'])

Calculating quick stats
{'number_of_nodes': 785, 'number_of_edges': 3661, 'average_degree': 9.327388535031847, 'density': 0.011897179253867152, 'clustering_coefficient': 0.2749767731563115}
Calculating community detection
0.7549565029663023


In [ ]:
pos = nx.kamada_kawai_layout(graph)
nx.draw(graph, pos, node_size=50)